__title__      = ML in Action Chapter7 Code(AdaBoost)
__author__     = wgj
__createDate__ = 2018-11-01 21:10:33

In [14]:
from numpy import *

""" 加载数据集 """
def loadDataSet(fileName): 
    """ 
    输入：fileName——文件名
    输出：1、dataMat——特征矩阵（m行xn列）
                 2、labelMat——标签向量（m行x1列）
    """
    numFeat = len(open(fileName).readline().split('\t'))      #  数据集列数
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr =[]                                                  # 保存一行训练数据
        curLine = line.strip().split('\t')              # 移除每行首尾空格并用tab分割数据
        for i in range(numFeat-1):                   # 每行22个数，前21个为特征
            lineArr.append(float(curLine[i]))
        dataMat.append(lineArr)
        labelMat.append(float(curLine[-1]))
    return dataMat,labelMat

""" 树桩分类 """
def stumpClassify(dataMatrix,dimen,threshVal,threshIneq):
    """ 
    输入：1、dataMatrix——特征矩阵（m行xn列）
                 2、dimen——特征索引
                 3、threshVal——阈值
                 4、threshIneq——比较方法（<，≥）
    输出：retArray——分类结果
    """
    retArray = ones((shape(dataMatrix)[0],1))                           # 初始化为全1向量
    if threshIneq == 'lt':                                                                  # ‘lt‘：less than，小于；‘gt‘：greater than，大于
        retArray[dataMatrix[:,dimen] <= threshVal] = -1.0        # 特征与阈值满足threshIneq关系的，分为-1类
    else:
        retArray[dataMatrix[:,dimen] > threshVal] = -1.0 
    return retArray
    
""" 构建单层决策树 """
def buildStump(dataArr,classLabels,D):
    """ 
    输入：1、dataArr——特征矩阵（m行xn列）
                 2、classLabels——标签向量
                 3、D——权向量
    输出：1、bestStump——最佳单层决策树（保存为字典）
                 2、minError——最小误差
                 3、bestClasEst——最佳分类结果
    """
    dataMatrix = mat(dataArr); labelMat = mat(classLabels).T
    m,n = shape(dataMatrix)
    numSteps = 10.0; bestStump = {}; bestClasEst = mat(zeros((m,1)))
    minError = inf       # 初始化误差和为正无穷
    for i in range(n):   # 遍历所有特征
        rangeMin = dataMatrix[:,i].min(); rangeMax = dataMatrix[:,i].max();    # 第i个特征大最小值和最大值
        stepSize = (rangeMax-rangeMin)/numSteps                                               # 步长
        for j in range(-1,int(numSteps)+1):                                                               # 在当前特征遍历步长
            for inequal in ['lt', 'gt']:  # 在大于、小于间切换
                threshVal = (rangeMin + float(j) * stepSize)                                        # 设置阈值
                predictedVals = stumpClassify(dataMatrix,i,threshVal,inequal)    # 依据当前特征进行分类
                errArr = mat(ones((m,1)))
                errArr[predictedVals == labelMat] = 0
                weightedError = D.T*errArr                         # 计算误差和
                #print "split: dim %d, thresh %.2f, thresh ineqal: %s, the weighted error is %.3f" % (i, threshVal, inequal, weightedError)
                if weightedError < minError:
                    minError = weightedError
                    bestClasEst = predictedVals.copy()
                    bestStump['dim'] = i                            # 单层决策树，树桩节点为（特征索引、阈值和比较方法）
                    bestStump['thresh'] = threshVal
                    bestStump['ineq'] = inequal
    return bestStump,minError,bestClasEst

""" 训练AdaBoost """
def adaBoostTrainDS(dataArr,classLabels,numIt=40):
    """ 
    输入：1、dataArr——特征矩阵（m行xn列）
                 2、classLabels——标签向量
                 3、numIt——迭代次数
    输出：1、weakClassArr——分类器参数
                 2、aggClassEst——每个数据点的类别估计累计值
    """
    weakClassArr = []
    m = shape(dataArr)[0]
    D = mat(ones((m,1))/m)   # 初始化所有权重为1/m
    aggClassEst = mat(zeros((m,1)))
    for i in range(numIt):
        bestStump,error,classEst = buildStump(dataArr,classLabels,D)    # 构建单层决策树 
        alpha = float(0.5*log((1.0-error)/max(error,1e-16)))                         # 计算alpha
        bestStump['alpha'] = alpha 
        weakClassArr.append(bestStump)               # 保存分类器参数
        expon = multiply(-1*alpha*mat(classLabels).T,classEst)            #  -alpha×yi×Gm
        D = multiply(D,exp(expon))                              # 更新权向量
        D = D/D.sum()
        #  计算所有分类器的误差和，若为0，结束循环
        aggClassEst += alpha*classEst              # 基本分类器的线性组合
        aggErrors = multiply(sign(aggClassEst) != mat(classLabels).T,ones((m,1)))
        errorRate = aggErrors.sum()/m
        print("total error: ",errorRate) 
        if errorRate == 0.0: break
    return weakClassArr,aggClassEst

""" 使用AdaBoost进行分类"""
def adaClassify(datToClass,classifierArr):
    """ 
    输入：1、datToClass——待分类样本
                 2、classifierArr——AdaBoost分类器
    输出：sign(aggClassEst)——
    """
    dataMatrix = mat(datToClass)
    m = shape(dataMatrix)[0]
    aggClassEst = mat(zeros((m,1)))
    for i in range(len(classifierArr)):
        classEst = stumpClassify(dataMatrix,classifierArr[i]['dim'],\
                                 classifierArr[i]['thresh'],\
                                 classifierArr[i]['ineq'])                                                 # 使用AdaBoost分类器信息进行决策树分类
        aggClassEst += classifierArr[i]['alpha']*classEst
#         print(aggClassEst) 
    return sign(aggClassEst)

从疝气病马预测死亡率

In [12]:
dataArr, labelArr = loadDataSet('horseColicTraining2.txt')
classifierArr, aggClassEst = adaBoostTrainDS(dataArr, labelArr, 10)

total error:  0.2842809364548495
total error:  0.2842809364548495
total error:  0.24749163879598662
total error:  0.24749163879598662
total error:  0.25418060200668896
total error:  0.2408026755852843
total error:  0.2408026755852843
total error:  0.22073578595317725
total error:  0.24749163879598662
total error:  0.23076923076923078


In [15]:
testArr, testLabelArr = loadDataSet('horseColicTest2.txt')
prediction = adaClassify(testArr,classifierArr)

In [27]:
errArr = mat(ones((len(testLabelArr),1)))
errRate = errArr[prediction!=mat(testLabelArr).T].sum()/len(testLabelArr)
print('错误率为：',errRate)

错误率为： 0.23880597014925373
